In [1]:
%load_ext autoreload

%autoreload
    
import numpy as np
import respy as rp
import pandas as pd

/home/maxbl/Uni/Robustness/structRobustness/sandbox/submodules/respy/respy/_numba.py:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.cgutils', please update to use 'numba.core.cgutils' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba import cgutils


In [2]:
process_name = "health_shock"
process_states = ["good", "bad"]

In [3]:
df_exog = pd.DataFrame(
    columns=process_states,
    index=["sick_and_old", "sick_and_young", "healthy_and_old", "healthy_and_young"],
    data=[[0.1, 0.9],
          [0.1, 0.9],
          [0.1, 0.9],
          [0.1, 0.9]],
    dtype=float
)
df_exog

,good,bad
sick_and_old,0.1,0.9
sick_and_young,0.1,0.9
healthy_and_old,0.1,0.9
healthy_and_young,0.1,0.9


In [4]:
params_exog, covs_exog = rp.parse_transition_matrix_for_exogenous_processes(df_exog, process_name)

In [5]:
params_exog

value
category                            name                       
exogenous_process_health_shock_good sick_and_old      -2.302585
                                    sick_and_young    -2.302585
                                    healthy_and_old   -2.302585
                                    healthy_and_young -2.302585
exogenous_process_health_shock_bad  sick_and_old      -0.105361
                                    sick_and_young    -0.105361
                                    healthy_and_old   -0.105361
                                    healthy_and_young -0.105361

In [6]:
covs_exog

{'sick_and_old': '?',
 'sick_and_young': '?',
 'healthy_and_old': '?',
 'healthy_and_young': '?'}

In [7]:
covs_exog['sick'] = "health_shock == 'bad'"
covs_exog['healthy'] = "health_shock == 'good'"
covs_exog['sick_and_old'] = "health_shock == 'bad' & old"
covs_exog['sick_and_young'] = "health_shock == 'bad' & young"
covs_exog['healthy_and_old'] = "health_shock == 'good' & old"
covs_exog['healthy_and_young'] = "health_shock == 'good' & young"
covs_exog

{'sick_and_old': "health_shock == 'bad' & old",
 'sick_and_young': "health_shock == 'bad' & young",
 'healthy_and_old': "health_shock == 'good' & old",
 'healthy_and_young': "health_shock == 'good' & young",
 'sick': "health_shock == 'bad'",
 'healthy': "health_shock == 'good'"}

In [8]:
covs_exog = {**covs_exog, "age": "16 + period",
                        "old": "age > 30",
                        "young": "age <= 30"}
covs_exog

{'sick_and_old': "health_shock == 'bad' & old",
 'sick_and_young': "health_shock == 'bad' & young",
 'healthy_and_old': "health_shock == 'good' & old",
 'healthy_and_young': "health_shock == 'good' & young",
 'sick': "health_shock == 'bad'",
 'healthy': "health_shock == 'good'",
 'age': '16 + period',
 'old': 'age > 30',
 'young': 'age <= 30'}

In [9]:
params, options = rp.get_example_model("kw_94_one", with_data=False)

In [10]:
params

value  \
category                name                                   
delta                   delta                         0.9500   
wage_a                  constant                      9.2100   
                        exp_edu                       0.0380   
                        exp_a                         0.0330   
                        exp_a_square                 -0.0005   
                        exp_b                         0.0000   
                        exp_b_square                  0.0000   
wage_b                  constant                      8.4800   
                        exp_edu                       0.0700   
                        exp_b                         0.0670   
                        exp_b_square                 -0.0010   
                        exp_a                         0.0220   
                        exp_a_square                 -0.0005   
nonpec_edu              constant                      0.0000   
                        at_least_twelve_exp_edu       0.0000   
                        not_edu_last_period       -4000.0000   
nonpec_home             constant                  17750.0000   
shocks_sdcorr           sd_a                          0.2000   
                        sd_b                          0.2500   
                        sd_edu                     1500.0000   
                        sd_home                    1500.0000   
                        corr_b_a                      0.0000   
                        corr_edu_a                    0.0000   
                        corr_edu_b                    0.0000   
                        corr_home_a                   0.0000   
                        corr_home_b                   0.0000   
                        corr_home_edu                 0.0000   
lagged_choice_1_edu     probability                   1.0000   
initial_exp_edu_10      probability                   1.0000   
maximum_exp             edu                          20.0000   
inadmissibility_penalty inadmissibility_penalty -400000.0000   

                                                                                           comment  
category                name                                                                        
delta                   delta                                                      discount factor  
wage_a                  constant                                               log of rental price  
                        exp_edu                          return to an additional year of schooling  
                        exp_a                                     return to same sector experience  
                        exp_a_square                   return to same sector, quadratic experience  
                        exp_b                                    return to other sector experience  
                        exp_b_square                  return to other sector, quadratic experience  
wage_b                  constant                                               log of rental price  
                        exp_edu                          return to an additional year of schooling  
                        exp_b                                     return to same sector experience  
                        exp_b_square                   return to same sector, quadratic experience  
                        exp_a                                    return to other sector experience  
                        exp_a_square                  return to other sector, quadratic experience  
nonpec_edu              constant                            constant reward for choosing education  
                        at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                        not_edu_last_period                        reward for going back to school  
nonpec_home             constant                         constant reward of non-market alternative  
shocks_sdcorr          

In [11]:
options["covariates"] = {**options["covariates"], **covs_exog}
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 3,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'sick_and_old': "health_shock == 'bad' & old",
  'sick_and_young': "health_shock == 'bad' & young",
  'healthy_and_old': "health_shock == 'good' & old",
  'healthy_and_young': "health_shock == 'good' & young",
  '

In [12]:
params.loc[("wage_a", "sick_and_old"), "value"] = -0.2
params.loc[("wage_a", "healthy_and_old"), "value"] = 0.2
params.loc[("wage_b", "sick_and_young"), "value"] = -0.2
params.loc[("wage_b", "healthy_and_young"), "value"] = 0.2
params.loc[("nonpec_edu", "sick"), "value"] = -0.1
params.loc[("nonpec_home", "sick"), "value"] = 5
params.loc[("observable_health_shock_good", "probability"), "value"] = 1
params.loc[("observable_health_shock_bad", "probability"), "value"] = 0
params_final = pd.concat([params, params_exog])

In [13]:
params_final

value  \
category                            name                                     
delta                               delta                         0.950000   
wage_a                              constant                      9.210000   
                                    exp_edu                       0.038000   
                                    exp_a                         0.033000   
                                    exp_a_square                 -0.000500   
                                    exp_b                         0.000000   
                                    exp_b_square                  0.000000   
wage_b                              constant                      8.480000   
                                    exp_edu                       0.070000   
                                    exp_b                         0.067000   
                                    exp_b_square                 -0.001000   
                                    exp_a                         0.022000   
                                    exp_a_square                 -0.000500   
nonpec_edu                          constant                      0.000000   
                                    at_least_twelve_exp_edu       0.000000   
                                    not_edu_last_period       -4000.000000   
nonpec_home                         constant                  17750.000000   
shocks_sdcorr                       sd_a                          0.200000   
                                    sd_b                          0.250000   
                                    sd_edu                     1500.000000   
                                    sd_home                    1500.000000   
                                    corr_b_a                      0.000000   
                                    corr_edu_a                    0.000000   
                                    corr_edu_b                    0.000000   
                                    corr_home_a                   0.000000   
                                    corr_home_b                   0.000000   
                                    corr_home_edu                 0.000000   
lagged_choice_1_edu                 probability                   1.000000   
initial_exp_edu_10                  probability                   1.000000   
maximum_exp                         edu                          20.000000   
inadmissibility_penalty             inadmissibility_penalty -400000.000000   
wage_a                              sick_and_old                 -0.200000   
                                    healthy_and_old               0.200000   
wage_b                              sick_and_young               -0.200000   
                                    healthy_and_young             0.200000   
nonpec_edu                          sick                         -0.100000   
nonpec_home                         sick                          5.000000   
observable_health_shock_good        probability                   1.000000   
observable_health_shock_bad         probability                   0.000000   
exogenous_process_health_shock_good sick_and_old                 -2.302585   
                                    sick_and_young               -2.302585   
                                    healthy_and_old              -2.302585   
                                    healthy_and_young            -2.302585   
exogenous_process_health_shock_bad  sick_and_old                 -0.105361   
                                    sick_and_young               -0.105361   
                                    healthy_and_old              -0.105361   
                                    healthy_and_young            -0.105361   

                                                                                                       comment  
category                            name                                                                        
delta               

In [14]:
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 3,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'sick_and_old': "health_shock == 'bad' & old",
  'sick_and_young': "health_shock == 'bad' & young",
  'healthy_and_old': "health_shock == 'good' & old",
  'healthy_and_young': "health_shock == 'good' & young",
  '

In [15]:
options["covariates"]

{'constant': '1',
 'exp_a_square': 'exp_a ** 2',
 'exp_b_square': 'exp_b ** 2',
 'at_least_twelve_exp_edu': 'exp_edu >= 12',
 'not_edu_last_period': "lagged_choice_1 != 'edu'",
 'sick_and_old': "health_shock == 'bad' & old",
 'sick_and_young': "health_shock == 'bad' & young",
 'healthy_and_old': "health_shock == 'good' & old",
 'healthy_and_young': "health_shock == 'good' & young",
 'sick': "health_shock == 'bad'",
 'healthy': "health_shock == 'good'",
 'age': '16 + period',
 'old': 'age > 30',
 'young': 'age <= 30'}

In [16]:
simulate = rp.get_simulate_func(params_final, options)
df = simulate(params)

> /home/maxbl/Uni/Robustness/structRobustness/sandbox/submodules/respy/respy/simulate.py(69)get_simulate_func()
-> df = _process_data_for_simulation(
(Pdb) period_indices
*** NameError: name 'period_indices' is not defined
(Pdb) l
 64  	
 65  	    n_simulation_periods, options = _harmonize_simulation_arguments(
 66  	        method, df, n_simulation_periods, options
 67  	    )
 68  	    breakpoint()
 69  ->	    df = _process_data_for_simulation(
 70  	        df, method, n_simulation_periods, options, optim_paras
 71  	    )
 72  	
 73  	    solve = get_solve_func(params, options)
 74  	
(Pdb) options
{'estimation_draws': 200, 'estimation_seed': 500, 'estimation_tau': 500, 'interpolation_points': -1, 'simulation_agents': 1000, 'simulation_seed': 132, 'solution_draws': 500, 'solution_seed': 3, 'core_state_space_filters': ['period > 0 and exp_a == period and lagged_choice_1 != 0', 'period > 0 and exp_b == period and lagged_choice_1 != 1', 'period > 0 and exp_edu == period and lagged_cho

(Pdb) period_indices
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0,

BdbQuit: 

In [ ]:
df["Health_Shock"].value_counts()

In [ ]:
df.columns.values